# The purpose of this Sprint

* Know the practical flow of machine learning
* Complete a model with high generalization performance

#[Problem 1] Cross Validation

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# loading the csv of the dataset
df = pd.read_csv('application_train.csv')
df = df.select_dtypes('number')

# cleaning the dataset by filling the empy data(null)
cleaned_df = df.fillna(0)

# get only existing data with no missing values
cleaned_df = cleaned_df[cleaned_df.columns[~cleaned_df.isnull().all()]]

# separating them into variables
y = cleaned_df['TARGET']
X = cleaned_df.drop(['TARGET'], axis=1)

X = X.to_numpy()

kf = KFold(n_splits=2)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [153756 153757 153758 ... 307508 307509 307510] TEST: [     0      1      2 ... 153753 153754 153755]
TRAIN: [     0      1      2 ... 153753 153754 153755] TEST: [153756 153757 153758 ... 307508 307509 307510]


Standardizing the data

In [2]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)

#[Problem 2] Grid search

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# checking which model and params are best
model_params = {
    'random_forest':{
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1,5,10]
        }
    },
    'logic_regression':{
        'model': LogisticRegression(solver="liblinear",multi_class="auto"),
        'params': {
            'C': [1,5,10]
        }
    }
}

# defining an array to store the scores
scores = []

for model_name,mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'], return_train_score=False)
    clf.fit(X_train_trans,y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

best_model_params = pd.DataFrame(scores,columns=['model','best_score','best_params'])
best_model_params

,model,best_score,best_params
0,random_forest,0.915789,{'n_estimators': 10}
1,logic_regression,0.918364,{'C': 1}


#[Problem 3] Survey from Kaggle Notebooks

* Hyperparameter Tuning using Grid search
* Gradient Boosting Machine
* Using one type of data
* Early stopping

#[Problem 4] Creating a model with high generalization performance

In [4]:
import lightgbm as lgb

# creating an instance of the model
model = lgb.LGBMClassifier()

# save the default params
default_params = model.get_params()

# number of folds
N_FOLDS = 5

# creating a dataset
train_set = lgb.Dataset(data = X_train)

# Cross validation results when avoid overfitting
cv_results = lgb.cv(default_params, train_set, num_boost_round = 10000, early_stopping_rounds = 100, metrics = 'auc', nfold = N_FOLDS, seed = 42)

# displaying the results
print('The maximum validation ROC AUC was: {:.5f}.'.format(cv_results['auc-mean'][-1]))
print('The optimal number of boosting rounds (estimators) was {}.'.format(len(cv_results['auc-mean'])))

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:430: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:741: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


The maximum validation ROC AUC was: 1.00000.
The optimal number of boosting rounds (estimators) was 1.


* I have imported the whole dataset
* I created a subset of only numbers
* I split the data using kfold
* I used gridsearchCV to find the best model and params to fine tune my classiffiers
* After finding the results i tested the LGBM classifier and checked it ROC which is good

**All of the techniques learnt are used through out the whole task in every problem**

#[Problem 5] Final model selection

In [5]:
# loading the csv of the test dataset
test_df = pd.read_csv('application_test.csv')

# cleaning the dataset by removing the empy data(null)
test_cleaned_df = test_df.fillna(0)

# separating them into variables
test_X = test_cleaned_df.select_dtypes('number')

# standardizing the data
test_scaler = StandardScaler()
test_X_test_trans = scaler.fit_transform(test_X)

# predicting
test_reg_pred = clf.predict(test_X_test_trans)

kgl_submission = pd.concat([test_df['SK_ID_CURR'], pd.Series(test_reg_pred, name='TARGET')], axis=1)
kgl_submission.to_csv(' ', index=False)